In [1]:
print("test")

test


In [5]:
# Simple Ollama LLM smoke test (single response + chat session)
# Prereq: `ollama serve` running locally, and model exists (e.g. `qwen3:4b`).

from memory_base.llm.base import LLMClient
from memory_base.llm.providers import OllamaLLM

# MODEL = "qwen3:4b"
MODEL = "qwen2.5:3b"
MESSAGE = "用一句话介绍一下你自己。"

provider = OllamaLLM(base_url="http://localhost:11434", timeout_s=60.0)
llm = LLMClient(llm=provider, model=MODEL)

print("=== single respond ===")
resp = llm.respond(MESSAGE)
print(resp.text)

print("\n=== chat session ===")
MESSAGE = "What's the capital of France?"
session = llm.start_chat()
print("user:", MESSAGE)
r1 = session.send(MESSAGE)
print("assistant:", r1.text)
assert("Paris" in r1.text)

followup = "What about Germany?"
print("user:", followup)
r2 = session.send(followup)
print("assistant:", r2.text)
assert("Berlin" in r2.text)

print("\n(history length)", len(session.history))



=== single respond ===
我是一个由阿里云开发的大规模语言模型，旨在通过理解和模拟人类对话来提供帮助和服务。

=== chat session ===
user: What's the capital of France?
assistant: The capital of France is Paris.
user: What about Germany?
assistant: The capital of Germany is Berlin.

(history length) 4


In [3]:
# qwen2.5:3b: extract NEI from a list of hand-written life-task descriptions
# Prereq: `ollama serve` running locally, and model exists (e.g. `qwen2.5:3b`).

from memory_base.adapters import TextToNEIAdapter
from memory_base.llm.base import LLMClient
from memory_base.llm.providers import OllamaLLM

MODEL = "qwen2.5:3b"
# MODEL = "qwen3:4b"
provider = OllamaLLM(base_url="http://localhost:11434", timeout_s=60.0)
llm = LLMClient(llm=provider, model=MODEL)
adapter = TextToNEIAdapter(llm=llm)

texts = [
    # more natural, longer narrative (work-like)
    (
        "昨晚我想把一个证明/形式化相关的任务往前推一点，但一开始就被一个编译错误卡住了。"
        "当时我没有先把错误信息整理清楚，就凭感觉去调整 tactic 顺序、顺手补了一个辅助引理。"
        "改完以后，原来的报错没消失，反而又冒出一个新的 type mismatch，让定位更困难。"
        "现在回头看，关键问题是我没有先把状态固定下来：应该先最小化复现、记录 error 位置、弄清依赖链，"
        "然后做最小改动一条条验证。"
    ),
    # more natural, longer narrative (fitness)
    (
        "下班回家已经很晚了，我其实挺累，但还是想把今天的运动补上。"
        "家里器械不多，只有一副哑铃和瑜伽垫，时间也就剩半小时。"
        "我没怎么热身就直接开始做深蹲和推举，前几组还行，后面呼吸乱了、动作也开始变形，"
        "膝盖还隐隐有点不舒服，于是提前停下来了。"
        "我意识到问题不是意志力，而是节奏不对：下一次应该先热身、用轻重量把动作做稳，再逐步加量。"
    ),
    # more natural, longer narrative (house cleanup)
    (
        "周末我想着把家里收拾一下，但东西堆了挺久，光是想到要整理就有点头大。"
        "我从客厅角落那堆杂物开始，想着先挪出来再分类，结果越挪越多，地上到处都是。"
        "两个小时过去，垃圾袋还没装满，反而把自己搞得很烦。"
        "复盘后我发现自己每拿起一个东西都会犹豫要不要留，进度就被反复打断。"
        "下次我会把范围缩小到一次只处理一个抽屉，并用“扔/留/归位”三个箱子强制决策，定个 25 分钟闹钟到点就停。"
    ),
]

for i, text in enumerate(texts, start=1):
    print(f"\n================ case {i} ================")
    print("=== input text ===")
    print(text)

    nei = adapter.ingest(text)
    print("\n=== extracted NEI ===")
    print("label:", nei.label)
    print("situation:", nei.situation)
    print("goal:", nei.goal)
    print("attempt:", nei.attempt)
    print("result:", nei.result)
    print("reflection:", nei.reflection)




================ case 1 ================
=== input text ===
昨晚我想把一个证明/形式化相关的任务往前推一点，但一开始就被一个编译错误卡住了。当时我没有先把错误信息整理清楚，就凭感觉去调整 tactic 顺序、顺手补了一个辅助引理。改完以后，原来的报错没消失，反而又冒出一个新的 type mismatch，让定位更困难。现在回头看，关键问题是我没有先把状态固定下来：应该先最小化复现、记录 error 位置、弄清依赖链，然后做最小改动一条条验证。

=== extracted NEI ===
label: failure
situation: 当时我在尝试把一个证明任务往前推进，遇到了一个编译错误。
goal: 修复并解决代码中的类型不匹配问题。
attempt: 我开始根据感觉调整 tactic 顺序，并添加了一个辅助引理。修改完后，原来的错误没有消失，反而又出现了一个新的 type mismatch 错误，使得定位问题变得更困难。
result: 目前，我发现关键问题是我在尝试修复之前并没有正确固定代码的状态：我应该首先最小化复现问题、记录 error 位置并弄清楚依赖链，然后逐一验证修改的效果。
reflection: None

================ case 2 ================
=== input text ===
下班回家已经很晚了，我其实挺累，但还是想把今天的运动补上。家里器械不多，只有一副哑铃和瑜伽垫，时间也就剩半小时。我没怎么热身就直接开始做深蹲和推举，前几组还行，后面呼吸乱了、动作也开始变形，膝盖还隐隐有点不舒服，于是提前停下来了。我意识到问题不是意志力，而是节奏不对：下一次应该先热身、用轻重量把动作做稳，再逐步加量。

=== extracted NEI ===
label: unknown
situation: 下班回家已经很晚了，我其实挺累的，但还是想把今天的运动补上。家里只有两件工具：一副哑铃和瑜伽垫，时间也就剩半小时。
goal: 完成预定的家庭锻炼计划
attempt: 开始进行深蹲和推举练习，在前几组时动作还算流畅且没有明显的不适感；但随着呼吸的混乱和动作变形，膝盖也感到隐隐作痛，意识到问题并非由于意志力而是缺乏正确的热身步骤。
result: 